Al abrir el JSON en VS hemos observado que  tiene una estructura anidada  Cada clave (ejemplo: "3", "4", "5") tiene una lista de datos. Cada elemento de esa lista es un diccionario con una clave y un valor: t_id con valor una imagen y duracion con un numero de minutos. Lo que queremos hacer es convertir este jason en un Dataframe, una tabla, que recoja esta informacion y con la que podamos trabajar en Pandas. Los Headers de las columnas van a ser las claves, y las filas van a ser los valores.

In [2]:
#Importar Librerias
import pandas as pd
import json
import numpy as np

# COmo el json es complejo lo vamos a leer tal cual es, como un diccionario

#leer json de manera estandard - hemos encontrado ese trozo de codigo en bibliografia

with open("/content/emocion.json.json", "r", encoding="utf-8") as file:
    datos = json.load(file)

# Ver estructura y efectivamente es un diccionario
print(type(datos))

#creamos una lista vacia para almacenar los datos
datos_emocion =[]

#Cuando recorremos datos.items(), queremos que en el bucle for para cada imagen nos guarde el diccionario en la variable diccionario
#asi en la primera vuelta obtenemos
#imagen	"Training_10118481.jpg"
#diccionario	{"t_id": "Training_10118481.jpg", "emocion": "angry", "tiempo_recogida": 291}
for imagen , diccionario in datos.items():
   datos_emocion.append(diccionario)

df_emocion=pd.DataFrame(datos_emocion)
df_emocion.head(20)




<class 'dict'>


,t_id,emocion,tiempo_recogida
0,Training_10118481.jpg,angry,291
1,Training_10120469.jpg,angry,425
2,Training_10131352.jpg,angry,499
3,Training_10161559.jpg,angry,715
4,Training_1021836.jpg,angry,301
5,Training_10269675.jpg,angry,415
6,Training_10278738.jpg,angry,458
7,Training_10290703.jpg,angry,104
8,Training_10295477.jpg,angry,184
9,Training_10315441.jpg,angry,221


Limpiamos un poco a ver que conclusiones podemos sacar, la columa de tiempo de recogida

In [ ]:
df_emocion.describe()

,tiempo_recogida
count,35885.000000
mean,359.592197
std,207.274279
min,2.000000
25%,181.000000
50%,359.000000
75%,539.000000
max,719.000000


In [ ]:
#vamos a ordenar el DF for el tiempo de recogida de la foto, para ver que foto se tomo antes y cuando
df_emocion = df_emocion.sort_values(by="tiempo_recogida")
df_emocion.head(20)

,t_id,emocion,tiempo_recogida
3459,Training_88504568.jpg,angry,2
32448,PublicTest_87513106.jpg,happy,2
19901,Training_85511311.jpg,neutral,2
10035,Training_28136915.jpg,happy,2
20927,Training_1436609.jpg,sad,2
2481,Training_661131.jpg,angry,2
17967,Training_49862858.jpg,neutral,2
12984,Training_65944675.jpg,happy,2
17020,Training_32303287.jpg,neutral,2
10889,Training_39357416.jpg,happy,2


Nos dice el proyecto que el tiempo re recogida representa el momento que se toma la foto contando el numero de horas que han pasado desde las 7:00 del 1 de Septiembre 2022. Vamos a ver como creamos una nueva columa con la fecha y la hora de la foto

In [ ]:
primera_fecha=pd.Timestamp("7:00 09-01-2022")
df_emocion["Fecha y hora foto"]=primera_fecha + pd.to_timedelta(df_emocion["tiempo_recogida"], unit="h") # no se podia sumar directamente, usamos
#pd.to_timedelta() convierte valores numéricos (como enteros o flotantes) en intervalos de tiempo (timedelta), que luego se pueden sumar
df_emocion.head(5)


,t_id,emocion,tiempo_recogida,Fecha y hora foto
3459,Training_88504568.jpg,angry,2,2022-09-01 09:00:00
32448,PublicTest_87513106.jpg,happy,2,2022-09-01 09:00:00
19901,Training_85511311.jpg,neutral,2,2022-09-01 09:00:00
10035,Training_28136915.jpg,happy,2,2022-09-01 09:00:00
20927,Training_1436609.jpg,sad,2,2022-09-01 09:00:00


In [8]:
def resumen_general(df):
    resumen = pd.DataFrame({
        "Columnas": df.columns,
        "Tipo de Dato": df.dtypes,
        "Valores Únicos": df.nunique(),
        "Valores Nulos": df.isnull().sum(),
        "Porcentaje Nulos (%)": df.isnull().mean() * 100,
        "Total de registros duplicados":df.duplicated().sum()
    })
    return resumen

resumen = resumen_general(df_emocion)
print(resumen)

                        Columnas Tipo de Dato  Valores Únicos  Valores Nulos  \
t_id                        t_id       object           35885              0   
emocion                  emocion       object               7            351   
tiempo_recogida  tiempo_recogida        int64             718              0   

                 Porcentaje Nulos (%)  Total de registros duplicados  
t_id                         0.000000                              0  
emocion                      0.978125                              0  
tiempo_recogida              0.000000                              0  


In [9]:
from os import sep


# Ruta donde guardar el CSV en Drive
ruta_csv = "/content/id_visitante-emocion_limpio.csv"

# Guardar como CSV
df_emocion.to_csv(ruta_csv, index=False, encoding="utf-8", sep=";")
